<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Creates-the-Kaggle-dataset" data-toc-modified-id="Creates-the-Kaggle-dataset-1">Creates the Kaggle dataset</a></span></li><li><span><a href="#Saves-the-non-kaggle-version" data-toc-modified-id="Saves-the-non-kaggle-version-2">Saves the non kaggle version</a></span></li></ul></div>

In [1]:
%%javascript
$('#menubar').toggle();

<IPython.core.display.Javascript object>

In [66]:
import numpy as np
import pandas as pd

# Creates the Kaggle dataset

In [67]:
from utils import read_data, process_time

In [68]:
infos, items, orders = read_data("datasets/")

In [69]:
#orders.query("month == 6 and day == 16")["time"].dt.dayofyear

In [70]:
"""
orders["time"] = pd.to_datetime(orders["time"])
orders["month"] = orders["time"].dt.month
orders["day"] = orders["time"].dt.day

temp = orders.query("month == 6 and itemID == 158")
temp.groupby(["day", "itemID"]).sum()
"""

'\norders["time"] = pd.to_datetime(orders["time"])\norders["month"] = orders["time"].dt.month\norders["day"] = orders["time"].dt.day\n\ntemp = orders.query("month == 6 and itemID == 158")\ntemp.groupby(["day", "itemID"]).sum()\n'

In [71]:
process_time(orders)

In [72]:
val_data = orders.query("group_backwards == 1")
val_data = val_data.groupby(["itemID"])["order"].sum()
val_data.head()

itemID
1      3
3    140
4    145
5      1
7      1
Name: order, dtype: int64

In [77]:
# Make sure the itemIDs are unique ints from 1 to len()+1
(items["itemID"].unique() == np.arange(1, len(items)+1)).all()

True

In [96]:
# Fill in with zeros
final = pd.DataFrame(columns=["demandPrediction"], index=np.arange(1, len(items)+1))
final["demandPrediction"] = 0
# Add substitute non zeros
final.loc[val_data.index] = val_data.values.reshape(-1, 1)

In [97]:
final.head()

,demandPrediction
1,3
2,0
3,140
4,145
5,1


In [98]:
# Add weight
#infos.set_index("itemID", inplace=True)
final["simulationPrice"] = infos["simulationPrice"]

In [99]:
final.head()

,demandPrediction,simulationPrice
1,3,3.43
2,0,9.15
3,140,14.04
4,145,14.10
5,1,7.48


# Saves the non kaggle version

In [100]:
final.to_csv("../autotest.csv", header=["demandPrediction", "simulationPrice"],
                                     index_label="itemID", sep="|")